In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import napari_sparrow as nas
from spatialdata import read_zarr
import os
import scanpy as sc
from spatialdata import SpatialData
from typing import Dict, List, Optional, Tuple
from napari_sparrow.table._table import _back_sdata_table_to_zarr
from napari_sparrow.table._annotation import _annotate_celltype
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
import anndata as ad
from random import sample 
import seaborn as sns


the value of the environment variable BASIC_DCT_BACKEND is not in ["JAX","SCIPY"]


In [2]:
df_atlas_percentages = pd.read_csv("/home/wout/Documents/Thesis_lokaal/Mouse_Liver_Resolve_Data/basic_annotation_percentage_atlas.csv", index_col=0)

In [3]:
def make_umap(sdata,n_PCAs,n_neighbors):
    sc.pp.neighbors(sdata.table, n_neighbors=n_neighbors, n_pcs=n_PCAs)
    sc.tl.umap(sdata.table)
    sdata.table.uns['umap_'+str(n_PCAs)+'_'+str(n_neighbors)] = sdata.table.uns['umap']

In [4]:
def visualize_classification(sdata,classification_name,umap_name,path_mg,cell_type_annotation=True,plot_umap=True,plot_dot_plot=True,plot_rank_genes_groups=True,plot_image=True):
    # Plot the UMAP
    if plot_umap:
        sdata.table.uns['umap'] = sdata.table.uns[umap_name]
        sc.pl.umap(sdata.table,color=[classification_name])
    
    # Give the cell type proportions
    df_prediction = pd.DataFrame(sdata.table.obs[classification_name].value_counts(normalize=True))
    df_prediction.sort_index(inplace=True)
    df_pred = df_prediction * 100
    if cell_type_annotation:
        df_atlas = df_atlas_percentages * 100
        df_atlas.loc['Unknown'] = 100-df_atlas.sum()
        print(df_atlas.round(6).abs())
        print(df_pred.round(6).abs())
    else:
        print(df_pred.round(6).abs())

    # Plot expression of the marker genes for each cluster
    if plot_dot_plot:
        make_dot_plot(sdata,path_mg,classification_dot_plot=classification_name)

    # Plot the highly differential genes for each cluster
    if plot_rank_genes_groups:
        sdata.table.uns['log1p']["base"] = None
        sc.tl.rank_genes_groups(sdata.table, groupby=classification_name,key=classification_name+'_rank_genes')
        sc.pl.rank_genes_groups(sdata.table, n_genes=8, sharey=False, show=False)
    
    # Plot the image with the cells
    if plot_image:
        nas.pl.plot_shapes(sdata,column=classification_name,img_layer='clahe',shapes_layer = "segmentation_mask_boundaries")

In [5]:
# make dot plot
def make_dot_plot(sdata,path_mg,classification_dot_plot):
    marker_genes = pd.read_csv(path_mg, sep=',', index_col=0)
    cell_types = marker_genes.columns 
    df_backup = sdata.table.var.copy(deep=True)     
    all_genes = sdata.table.var.index.str.capitalize()
    for gene in all_genes:
        if gene not in marker_genes.index:
            marker_genes.loc[gene] = [0]*len(marker_genes.columns)  
    marker_genes.sort_index(inplace=True) 
    sdata.table.var = sdata.table.var.join(marker_genes)
    sdata.table.var['sum'] = sdata.table.var.iloc[:,len(sdata.table.var.columns)-len(cell_types):len(sdata.table.var.columns)].sum(axis=1)
    positions_labels_dict = {}
    for cell_type in cell_types:
        positions = np.where(sdata.table.var[cell_type]==1)[0]
        for p in positions:
            if (p,p) in positions_labels_dict:
                positions_labels_dict[(p,p)] = positions_labels_dict[(p,p)] + '_' + cell_type.lower()
            else:
                positions_labels_dict[(p,p)] = cell_type.lower()
    keys = positions_labels_dict.keys()
    values = positions_labels_dict.values()    
    sc.pl.dotplot(sdata.table,var_names=sdata.table.var_names,groupby=classification_dot_plot,dendrogram=True,var_group_positions=list(keys),var_group_labels=list(values))    
    sdata.table.var = df_backup
    

In [6]:
def score_genes_bins(
    sdata: SpatialData,
    path_marker_genes: str,
    bins: int = 25,
    delimiter=",",
    row_norm: bool = False,
    repl_columns: Optional[Dict[str, str]] = None,
    del_celltypes: Optional[List[str]] = None,
    input_dict=False,
) -> Tuple[dict, pd.DataFrame]:
    """
    The function loads marker genes from a CSV file and scores cells for each cell type using those markers
    using scanpy's score_genes function.
    Marker genes can be provided as a one-hot encoded matrix with cell types listed in the first row, and marker genes in the first column;
    or in dictionary format. The function further allows replacements of column names and deletions of specific marker genes.

    Parameters
    ----------
    sdata : SpatialData
        Data containing spatial information.
    path_marker_genes : str
        Path to the CSV file containing the marker genes.
        CSV file should be a one-hot encoded matrix with cell types listed in the first row, and marker genes in the first column.
    bins : int, optional
        Number of bins to use for the sc.tl.score_genes function, default is 25.
    delimiter : str, optional
        Delimiter used in the CSV file, default is ','.
    row_norm : bool, optional
        Flag to determine if row normalization is applied, default is False.
    repl_columns : dict, optional
        Dictionary containing cell types to be replaced. The keys are the original cell type names and
        the values are their replacements.
    del_celltypes : list, optional
        List of cell types to be deleted from the list of possible cell type candidates.
        Cells are scored for these cell types, but will not be assigned a cell type from this list.
    input_dict : bool, optional
        If True, the marker gene list from the CSV file is treated as a dictionary with the first column being
        the cell type names and the subsequent columns being the marker genes for those cell types. Default is False.

    Returns
    -------
    dict
        Dictionary with cell types as keys and their respective marker genes as values.
    pd.DataFrame
        Index:
            cells: The index corresponds to indivdual cells ID's.
        Columns:
            celltypes (as provided via the markers file).
        Values:
            Score obtained using scanpy's score_genes function for each celltype and for each cell.

    Notes
    -----
    The cell type 'unknown_celltype' is reserved for cells that could not be assigned a specific cell type.

    """

    # Load marker genes from csv
    if input_dict:
        df_markers = pd.read_csv(
            path_marker_genes, header=None, index_col=0, delimiter=delimiter
        )
        df_markers = df_markers.T
        genes_dict = df_markers.to_dict("list")
        for i in genes_dict:
            genes_dict[i] = [x for x in genes_dict[i] if str(x) != "nan"]
    # Replace column names in marker genes
    else:
        df_markers = pd.read_csv(path_marker_genes, index_col=0, delimiter=delimiter)
        if repl_columns:
            for column, replace in repl_columns.items():
                df_markers.columns = df_markers.columns.str.replace(column, replace)

        # Create genes dict with all marker genes for every celltype
        genes_dict = {}
        for i in df_markers:
            genes = []
            for row, value in enumerate(df_markers[i]):
                if value > 0:
                    genes.append(df_markers.index[row])
            genes_dict[i] = genes

    assert (
        "unknown_celltype" not in genes_dict.keys()
    ), "Cell type 'unknown_celltype' is reserved for cells that could not be assigned a specific cell type"

    # Score all cells for all celltypes
    for key, value in genes_dict.items():
        try:
            sc.tl.score_genes(sdata.table, value, score_name=key,n_bins=bins) # W: key = cell type, value = list of markergenes of that cell type
        except ValueError:
            log.warning(
                f"Markergenes {value} not present in region, celltype {key} not found"
            )

    # Delete genes from marker genes and genes dict
    if del_celltypes:
        for gene in del_celltypes:
            if gene in df_markers.columns:
                del df_markers[gene]
            if gene in genes_dict.keys():
                del genes_dict[gene]

    sdata, scoresper_cluster = _annotate_celltype( 
        sdata=sdata,
        celltypes=df_markers.columns,
        row_norm=row_norm,
        celltype_column="annotation",
    )

    # add 'unknown_celltype' to the list of celltypes if it is detected.
    if "unknown_celltype" in sdata.table.obs["annotation"].cat.categories:
        genes_dict["unknown_celltype"] = []

    name_clustering = 'score_genes_' + str(bins)
    sdata.table.uns[name_clustering] = scoresper_cluster
    sdata.table.obs.rename(columns={'annotation': 'annotation_'+name_clustering}, inplace=True)
    sdata.table.obs.rename(columns={'Cleanliness': 'cleanliness_'+name_clustering}, inplace=True)
    del genes_dict['unknown_celltype']
    sdata.table.obs.drop(genes_dict.keys(), axis=1, inplace=True)
    cols = sdata.table.obs.columns.to_list()
    cols_new = cols[0:len(cols)-2]
    cols_new.append(cols[len(cols)-1])
    cols_new.append(cols[len(cols)-2])
    sdata.table.obs  = sdata.table.obs .reindex(columns=cols_new)

    _back_sdata_table_to_zarr(sdata)

    return genes_dict, scoresper_cluster

    

In [9]:
def own_score_genes(sdata,path_mg,norm_expr_var=False,min_score='Quantile',min_score_q=25,scale_score='MinMax',scale_score_q=1,suffix='')->pd.DataFrame: 
    # annotate each cell
    # method based on score_genes of scanpy but no bins and min max normalization of the scores per cell type
    # for each cell, a score is calculated for each cell type: 
    # sum of the expressions of the markers in the cell - sum of the mean expressions of the markers in all cells
    # our expression data does not need to be scaled anymore (norm_expr_var = False) because sc.pp.scale is already applied in Sparrow
    path_marker_genes = path_mg,
    marker_genes = pd.read_csv(path_marker_genes[0], sep=',',index_col=0)
    scores_cell_celltype = pd.DataFrame()
    cell_types = marker_genes.columns.tolist()
    matrix = sdata.table.to_df()
    # correct for the variance of the expression of each gene
    if norm_expr_var:
        matrix = matrix.div(matrix.std(axis=0))
    all_mean_expression = matrix.mean(axis=0)
    for cell_type in cell_types:
        scores_cells = []
        for i in range(matrix.shape[0]):
            score = 0 
            for gene in marker_genes[marker_genes[cell_type] > 0].index.tolist():
                score = score + (matrix[gene][i] - all_mean_expression[gene])*marker_genes[cell_type][gene]
            scores_cells.append(score)
        scores_cell_celltype[cell_type] = scores_cells

    # min score to obtain for a cell type, otherwise 'unknown' 
    if min_score == 'Zero':
        scores_cell_celltype_ok = scores_cell_celltype.copy(deep=True)
        scores_cell_celltype_ok[scores_cell_celltype_ok > 0] = True
        scores_cell_celltype_ok[scores_cell_celltype_ok != True] = False
    if min_score == 'Quantile':
        scores_cell_celltype_ok = scores_cell_celltype.copy(deep=True)
        scores_cell_celltype_ok[scores_cell_celltype_ok > scores_cell_celltype_ok.quantile(min_score_q/100)] = True
        scores_cell_celltype_ok[scores_cell_celltype_ok != True] = False
    if min_score == 'None':
        scores_cell_celltype_ok = scores_cell_celltype.copy(deep=True)
        scores_cell_celltype_ok[scores_cell_celltype_ok.round(6) == scores_cell_celltype_ok.round(6)] = True

    # scale scores per cell type to make them more comparable between cell types (because some cell types have more markers etc.) 
    if scale_score == 'MinMax':
        scores_cell_celltype = scores_cell_celltype.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
    if scale_score == 'ZeroMax':
        scores_cell_celltype = scores_cell_celltype.apply(lambda x: (x) / (np.max(x))) # (~ min max scaling with min = 0)
    if scale_score == 'Nmarkers':
        Nmarkers = marker_genes.sum(axis=0).to_list()
        scores_cell_celltype = scores_cell_celltype.div(Nmarkers)
    if scale_score == 'Robust':
        for cell_type in cell_types:
            if np.percentile(scores_cell_celltype[cell_type],scale_score_q) < np.percentile(scores_cell_celltype[cell_type],100-scale_score_q):
                scores_cell_celltype[cell_type] = (scores_cell_celltype[cell_type] - np.percentile(scores_cell_celltype[cell_type],scale_score_q))/(np.percentile(scores_cell_celltype[cell_type],100-scale_score_q)-np.percentile(scores_cell_celltype[cell_type],scale_score_q))
            else: # MinMax scaling if percentiles are equal 
                scores_cell_celltype[cell_type] = (scores_cell_celltype[cell_type]-np.min(scores_cell_celltype[cell_type]))/(np.max(scores_cell_celltype[cell_type])-np.min(scores_cell_celltype[cell_type]))
    if scale_score == 'Rank':
        for cell_type in cell_types:
            scores_cell_celltype[cell_type] = scores_cell_celltype[cell_type].rank(pct=True)
            
    # cell is annotated with the cell type with the highest score (+ this highest score is above min_score)
    scores_cell_celltype[scores_cell_celltype_ok == False] = np.nan
    sc_cell_cellt = scores_cell_celltype.idxmax(axis=1).to_dict()
    unknown_cells = [k for k, v in sc_cell_cellt.items() if pd.isnull(v)]
    # change the values of keys in list
    for i in unknown_cells:
        sc_cell_cellt[i] = 'Unknown'
    sc_cell_cellt = {str(k): v for k, v in sc_cell_cellt.items()}
    sdata.table.obs["annotation_own_score_genes"+suffix] = sc_cell_cellt.values()
    # cleanliness of each annotation is calculated
    max_scores = scores_cell_celltype.max(axis=1)
    second_scores = scores_cell_celltype.apply(lambda x: x.nlargest(2).values[-1], axis=1)
    cleanliness = (max_scores - second_scores) / ((max_scores + second_scores) / 2)
    sc_cell_cleanl = cleanliness.to_dict()
    for i in unknown_cells:
        sc_cell_cleanl[i] = 0
    sc_cell_cleanl = {str(k): v for k, v in sc_cell_cleanl.items()}
    sdata.table.obs["score_celltype_own_score_genes"+suffix] = max_scores.values
    sdata.table.obs["second_score_celltype_own_score_genes"+suffix] = second_scores.values
    sdata.table.obs["cleanliness_own_score_genes"+suffix] = sc_cell_cleanl.values()
    return scores_cell_celltype


In [ ]:
def make_umap_and_perform_leiden_annotation(sdata,path_mg,n_PCAs,n_neighbors,cluster_resolution,norm_expr_var=True,min_score='Quantile',min_score_q=25,scale_score='Robust',scale_score_q=1,clean_th=0.5)->pd.DataFrame:

    # make umap and do leiden clustering with scanpy functions
    make_umap(sdata,n_neighbors=n_neighbors,n_PCAs=n_PCAs)
    column_name = 'leiden_'+str(n_PCAs)+'_'+str(n_neighbors)+'_'+str(cluster_resolution)
    sc.tl.leiden(sdata.table,resolution=cluster_resolution,key_added=column_name)

    # annotate each leiden cluster
    # method based on marker genes and similar to 'own_score_genes' but leiden clusters annotated instead of individual cells
    # for each leiden cluster, a score is calculated for each cell type: 
    # sum of the mean expressions of the markers in leiden cluster - sum of mean expression of the markers in all cells
    n_clusters = np.unique(sdata.table.obs[column_name]).size
    leiden_mean_expression = {}
    for i in range(n_clusters):
        an_cluster = sdata.table[sdata.table.obs[column_name]==str(i)]
        daf = an_cluster.to_df().mean(axis=0)
        pd.DataFrame(daf)
        leiden_mean_expression[i] = daf
    if norm_expr_var:
        matrix = sdata.table.to_df()
        all_mean_expression = matrix.div(matrix.std(axis=0)).mean(axis=0)
        for i in range(n_clusters):
            leiden_mean_expression[i] = leiden_mean_expression[i].div(matrix.std(axis=0))
    else:
        all_mean_expression = sdata.table.to_df().mean(axis=0)
    path_marker_genes = path_mg,
    marker_genes = pd.read_csv(path_marker_genes[0], sep=',',index_col=0)
    scores_leiden_celltype = pd.DataFrame()
    cell_types = marker_genes.columns.tolist()
    for cell_type in cell_types:
        scores_clusters = []
        for i in range(n_clusters):
            score = 0 
            for gene in marker_genes[marker_genes[cell_type] == 1].index.tolist():
                score = score + (leiden_mean_expression[i][gene] - all_mean_expression[gene])
            scores_clusters.append(score)
        scores_leiden_celltype[cell_type] = scores_clusters
    
    # min score to obtain for a cell type, otherwise 'unknown' 
    if min_score == 'Zero':
        scores_leiden_celltype_ok = scores_leiden_celltype.copy(deep=True)
        scores_leiden_celltype_ok[scores_leiden_celltype_ok > 0] = True
        scores_leiden_celltype_ok[scores_leiden_celltype_ok != True] = False
    if min_score == 'Quantile':
        scores_leiden_celltype_ok = scores_leiden_celltype.copy(deep=True)
        scores_leiden_celltype_ok[scores_leiden_celltype_ok > scores_leiden_celltype_ok.quantile(min_score_q/100)] = True
        print(min_score_q/100)
        scores_leiden_celltype_ok[scores_leiden_celltype_ok != True] = False
    if min_score == 'None':
        scores_leiden_celltype_ok = scores_leiden_celltype.copy(deep=True)
        scores_leiden_celltype_ok[scores_leiden_celltype_ok.round(6) == scores_leiden_celltype_ok.round(6)] = True

    # scale scores per cell type to make them more comparable between cell types (because some cell types have more markers etc.) 
    if scale_score == 'MinMax':
        scores_leiden_celltype = scores_leiden_celltype.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
    if scale_score == 'ZeroMax':
        scores_leiden_celltype = scores_leiden_celltype.apply(lambda x: (x) / (np.max(x))) # (~ min max scaling with min = 0)
    if scale_score == 'Nmarkers':
        Nmarkers = marker_genes.sum(axis=0).to_list()
        scores_leiden_celltype = scores_leiden_celltype.div(Nmarkers)
    if scale_score == 'Robust':
        for cell_type in cell_types:
            if np.percentile(scores_leiden_celltype[cell_type],scale_score_q) < np.percentile(scores_leiden_celltype[cell_type],100-scale_score_q):
                scores_leiden_celltype[cell_type] = (scores_leiden_celltype[cell_type] - np.percentile(scores_leiden_celltype[cell_type],scale_score_q))/(np.percentile(scores_leiden_celltype[cell_type],100-scale_score_q)-np.percentile(scores_leiden_celltype[cell_type],scale_score_q))
            else: # MinMax scaling if percentiles are equal 
                scores_leiden_celltype[cell_type] = (scores_leiden_celltype[cell_type]-np.min(scores_leiden_celltype[cell_type]))/(np.max(scores_leiden_celltype[cell_type])-np.min(scores_leiden_celltype[cell_type]))
    if scale_score == 'Rank':
        for cell_type in cell_types:
            scores_leiden_celltype[cell_type] = scores_leiden_celltype[cell_type].rank(pct=True)

    # cluster is annotated with the cell type with the highest score (+ this highest score is above min_score)
    scores_leiden_celltype[scores_leiden_celltype_ok == False] = np.nan
    sc_leiden_cellt = scores_leiden_celltype.idxmax(axis=1).to_dict()
    unknown_clusters = [k for k, v in sc_leiden_cellt.items() if pd.isnull(v)]

    max_scores = scores_leiden_celltype.max(axis=1)
    second_scores = scores_leiden_celltype.apply(lambda x: x.nlargest(2).values[-1], axis=1)
    cleanl_per_cluster = (max_scores - second_scores) / ((max_scores + second_scores) / 2)
    third_scores = scores_leiden_celltype.apply(lambda x: x.nlargest(3).values[-1], axis=1)
    cleanl_per_cluster_extra = (max_scores - third_scores) / ((max_scores + third_scores) / 2)
    scores_draft = scores_leiden_celltype.copy(deep=True)
    for i in range(n_clusters):
        if cleanl_per_cluster[i] < clean_th:
            scores_draft.loc[i].at[scores_draft.idxmax(axis=1)[i]] = np.nan 
            sc_leiden_cellt[i] = sc_leiden_cellt[i] + '/' + scores_draft.idxmax(axis=1)[i]
            if cleanl_per_cluster_extra[i] < clean_th:
                scores_draft.loc[i].at[scores_draft.idxmax(axis=1)[i]] = np.nan 
                sc_leiden_cellt[i] = sc_leiden_cellt[i] + '/' + scores_draft.idxmax(axis=1)[i]
                sum = abs(max_scores[i] + second_scores[i] + third_scores[i])
                if max_scores[i] > 0:
                    p1 = round(100*max_scores[i]/sum)
                    p2 = round(100*second_scores[i]/sum)
                    p3 = round(100*third_scores[i]/sum)
                else:
                    p1 = round(100*(sum + max_scores[i])/(2*sum))
                    p2 = round(100*(sum + second_scores[i])/(2*sum))
                    p3 = round(100*(sum + third_scores[i])/(2*sum))
                sc_leiden_cellt[i] = sc_leiden_cellt[i] + '(' + str(p1) + '%/' + str(p2) + '%/' + str(p3) + '%)'
            else:
                sum = abs(max_scores[i] + second_scores[i])
                if max_scores[i] > 0:
                    p1 = round(100*max_scores[i]/sum)
                    p2 = round(100*second_scores[i]/sum)
                else:
                    p1 = round(100*(sum + max_scores[i])/sum)
                    p2 = round(100*(sum + second_scores[i])/sum)
                sc_leiden_cellt[i] = sc_leiden_cellt[i] + '(' + str(p1) + '%/' + str(p2) + '%)'
    # change the values of keys in list
    for i in unknown_clusters:
        sc_leiden_cellt[i] = 'unknown'
    sc_leiden_cellt = {str(k): v for k, v in sc_leiden_cellt.items()}
    sdata.table.obs["annotation_"+column_name]=sdata.table.obs[column_name] 
    sdata.table.obs["annotation_"+column_name].replace(list(sc_leiden_cellt.keys()),list(sc_leiden_cellt.values()), inplace=True)
    b = pd.DataFrame.from_dict(sc_leiden_cellt, orient='index')
    cell_type_leiden = {}
    cell_types = np.unique(b[0])
    for cell_type in cell_types:
        indices = b.index[b[0] == cell_type].tolist()
        cell_type_leiden[cell_type] = indices
    sdata.table.uns["mapping_cell_type_"+column_name] = cell_type_leiden
    # cleanliness of the annotation based on highest and second highest score
    sc_leiden_cleanl = cleanl_per_cluster.to_dict()
    for i in unknown_clusters:
        sc_leiden_cleanl[i] = 0
    sc_leiden_cleanl = {str(k): v for k, v in sc_leiden_cleanl.items()}
    sdata.table.obs["cleanliness_"+column_name]=sdata.table.obs[column_name] 
    sdata.table.obs["cleanliness_"+column_name].replace(list(sc_leiden_cleanl.keys()),list(sc_leiden_cleanl.values()), inplace=True)

    return scores_leiden_celltype



In [2]:
def plot_dendrogram(model,N_clusters,labels) -> dict:
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    R = dendrogram(linkage_matrix,truncate_mode='lastp',p=N_clusters,no_plot=True)   
    
    R2 = dendrogram(linkage_matrix,labels=labels,no_plot=True)  
    clusters = np.array(list(dict.fromkeys(R2["ivl"])))
    clusters = clusters.astype(str)

    # create a label dictionary
    temp = {R["leaves"][ii]: clusters[ii] + ' ' + R["ivl"][ii] for ii in range(len(R["leaves"]))}
    def llf(xx):
        return "{}".format(temp[xx])

    dendrogram(linkage_matrix,leaf_label_func=llf,leaf_rotation=60.,leaf_font_size=10.,truncate_mode='lastp',p=N_clusters)

    return R

In [ ]:
def KMeans_clustering(sdata,matrix_score_genes,N_clusters,suffix_name=''):
    kmeans = KMeans(n_clusters = N_clusters, n_init=10) # run 10 times with different centroid seeds
    kmeans_annotation = kmeans.fit_predict(matrix_score_genes)
    kmeans_annotation = kmeans_annotation.astype(str)
    sdata.table.obs["KMeans"+str(N_clusters)+suffix_name] = kmeans_annotation

In [ ]:
def Hierarchical_clustering(sdata,matrix_score_genes,N_clusters,suffix_name='',levels_dendrogram=4)-> dict:
    hier = AgglomerativeClustering(n_clusters=N_clusters,compute_distances=True)
    hierarchical = hier.fit(matrix_score_genes)
    hierarchical_annotation = hierarchical.fit_predict(matrix_score_genes)
    hierarchical_annotation = hierarchical_annotation.astype(str)
    sdata.table.obs["Hierarchical"+str(N_clusters)+suffix_name] = hierarchical_annotation
    R = plot_dendrogram(hierarchical,N_clusters=N_clusters,labels=hierarchical_annotation)
    return R

In [ ]:
def correlation_matrix_expression_marker_genes_of_2_cell_types(anndata,path_mg,type1,type2):
    df_mg = pd.read_csv(path_mg,index_col=0)
    gene_set1 = df_mg.index[df_mg[type1]==1].tolist()
    gene_set2 = df_mg.index[df_mg[type2]==1].tolist()
    df_genes = pd.DataFrame()
    overlap = [g for g in gene_set1 if g in gene_set2]
    gene_set1 = [g for g in gene_set1 if g not in overlap]
    gene_set2 = [g for g in gene_set2 if g not in overlap]
    for g in gene_set1:
        df_genes[g+' '+type1] = anndata.to_df()[g]
    for g in overlap:
        df_genes[g+' both'] = anndata.to_df()[g]
    for g in gene_set2:
        df_genes[g+' '+type2] = anndata.to_df()[g]
    sns.heatmap(df_genes.corr(method='pearson'))

In [1]:
def DEGs_between_2_sets_leiden_clusters_compared_to_markers(adata,name_cell_type1, putative_leiden_clusters_cell_type1, name_cell_type2, putative_leiden_clusters_cell_type2, path_mg)->dict:
    leidcl1 = [str(x) for x in putative_leiden_clusters_cell_type1]
    leidcl2 = [str(x) for x in putative_leiden_clusters_cell_type2]
    a = adata.obs['leiden']
    for n in leidcl1:
        a = a.replace(n,leidcl1[0])
    for n in leidcl2:
        a = a.replace(n,leidcl2[0])
    adata.obs['leiden_mod'] = a
    #adata.uns['log1p']["base"] = None
    sc.tl.rank_genes_groups(adata, groupby='leiden_mod', groups = [leidcl1[0],leidcl2[0]], method = 'wilcoxon')
    #sc.pl.rank_genes_groups(sdata.table, n_genes=99, sharey=False, show=False)
    genes = pd.DataFrame(adata.uns['rank_genes_groups']['names'])
    genes.rename(columns = {leidcl1[0]:'gene_'+name_cell_type1,leidcl2[0]:'gene_'+name_cell_type2},inplace=True)
    pvals_adj = pd.DataFrame(adata.uns['rank_genes_groups']['pvals_adj'])
    pvals_adj.rename(columns = {leidcl1[0]:'pval_adj_'+name_cell_type1,leidcl2[0]:'pval_adj_'+name_cell_type2},inplace=True)
    logf2 = pd.DataFrame(adata.uns['rank_genes_groups']['logfoldchanges'])
    logf2.rename(columns = {leidcl1[0]:'logf2_'+name_cell_type1,leidcl2[0]:'logf2_'+name_cell_type2},inplace=True)
    df = pd.concat([genes,pvals_adj,logf2],axis=1)
    df_ct1_vs_rest = df[['gene_'+name_cell_type1,'pval_adj_'+name_cell_type1,'logf2_'+name_cell_type1]]
    df_ct1_vs_rest = df_ct1_vs_rest[(df_ct1_vs_rest['pval_adj_'+name_cell_type1] < 0.01) & (df_ct1_vs_rest['logf2_'+name_cell_type1] > 0)]
    df_ct2_vs_rest = df[['gene_'+name_cell_type2,'pval_adj_'+name_cell_type2,'logf2_'+name_cell_type2]]
    df_ct2_vs_rest = df_ct2_vs_rest[(df_ct2_vs_rest['pval_adj_'+name_cell_type2] < 0.01) & (df_ct2_vs_rest['logf2_'+name_cell_type2] > 0)]

    sc.tl.rank_genes_groups(adata, groupby='leiden_mod', groups = [leidcl1[0]], reference = leidcl2[0], method = 'wilcoxon')
    #sc.pl.rank_genes_groups(sdata.table, n_genes = 99, sharey=False, show=False)
    genes = pd.DataFrame(adata.uns['rank_genes_groups']['names'])
    genes.rename(columns = {leidcl1[0]:'gene'},inplace=True)
    pvals_adj = pd.DataFrame(adata.uns['rank_genes_groups']['pvals_adj'])
    pvals_adj.rename(columns = {leidcl1[0]:'pval_adj'},inplace=True)
    logf2 = pd.DataFrame(adata.uns['rank_genes_groups']['logfoldchanges'])
    logf2.rename(columns = {leidcl1[0]:'logf2'},inplace=True)
    df_ct1_vs_ct2 = pd.concat([genes,pvals_adj,logf2],axis=1)
    df_ct1_vs_ct2 = df_ct1_vs_ct2[df_ct1_vs_ct2['pval_adj'] < 0.01]
    
    df_mg = pd.read_csv(path_mg,index_col=0)
    mg_ct1 = df_mg.index[df_mg[name_cell_type1]==1].tolist()
    mg_ct2 = df_mg.index[df_mg[name_cell_type2]==1].tolist()
    mg_overlap = [x for x in mg_ct1 if x in mg_ct2]
    df_overlap = df_ct1_vs_ct2[df_ct1_vs_ct2['gene'].isin(mg_overlap)]
    candidates_ct1 = df_overlap[df_overlap['logf2']>0]['gene'].to_list()
    candidates_ct2 = df_overlap[df_overlap['logf2']<0]['gene'].to_list()
    drop_ct2 = []
    if len(candidates_ct1)>0:
        reject1 = df_ct2_vs_rest[df_ct2_vs_rest['gene_'+name_cell_type2].isin(candidates_ct1)]['gene_'+name_cell_type2].to_list()
        drop_ct2 = [x for x in candidates_ct1 if x not in reject1]
    drop_ct1 = []
    if len(candidates_ct2)>0:
        reject2 = df_ct1_vs_rest[df_ct1_vs_rest['gene_'+name_cell_type1].isin(candidates_ct2)]['gene_'+name_cell_type1].to_list()
        drop_ct1 = [x for x in candidates_ct2 if x not in reject2]
    results = {'DEGs': df_ct1_vs_ct2, 'DEGs_'+name_cell_type1+'_vs_rest': df_ct1_vs_rest, 'DEGs_'+name_cell_type2+'_vs_rest': df_ct2_vs_rest, 'markers_'+name_cell_type1: mg_ct1, 'markers_'+name_cell_type2: mg_ct2, 'overlap_markers': mg_overlap, 'drop_'+name_cell_type1: drop_ct1, 'drop_'+name_cell_type2: drop_ct2}
    return results

In [ ]:
def DEGs_between_each_leiden_cluster_and_rest_compared_to_markers(adata,name_cell_types, putative_leiden_clusters_per_cell_type, path_mg)->dict:
    a = adata.obs['leiden']
    leidcl = []
    for putative_leiden_clusters in putative_leiden_clusters_per_cell_type:
        L = [str(x) for x in putative_leiden_clusters]
        for n in L:
            a = a.replace(n,L[0])
        leidcl.append(L[0])
    adata.obs['leiden_mod'] = a
    #adata.uns['log1p']["base"] = None
    sc.tl.rank_genes_groups(adata, groupby='leiden_mod', method = 'wilcoxon')
    #sc.pl.rank_genes_groups(sdata.table, n_genes=99, sharey=False, show=False)
    genes = pd.DataFrame(adata.uns['rank_genes_groups']['names'])
    pvals_adj = pd.DataFrame(adata.uns['rank_genes_groups']['pvals_adj'])
    logf2 = pd.DataFrame(adata.uns['rank_genes_groups']['logfoldchanges'])
    dict_df_ct_vs_rest = {}
    dict_ct_markers = {}
    dict_ct_pos_DEG_but_not_marker = {}
    df_mg = pd.read_csv(path_mg,index_col=0)
    i = 0
    for nr in leidcl:
        df_ct_vs_rest = pd.concat([genes[[nr]],pvals_adj[[nr]],logf2[[nr]]],axis=1)
        # change column names of df_ct_vs_rest
        df_ct_vs_rest.columns = ['gene','pvals_adj','logf2']
        df_ct_vs_rest = df_ct_vs_rest[df_ct_vs_rest['pvals_adj'] < 0.01]
        dict_df_ct_vs_rest[name_cell_types[i]] = df_ct_vs_rest
        markers_ct = df_mg.index[df_mg[name_cell_types[i]]==1].tolist()
        dict_ct_markers[name_cell_types[i]] = markers_ct
        pos_DEGs_but_no_mg = df_ct_vs_rest[(~df_ct_vs_rest['gene'].isin(markers_ct)) & (df_ct_vs_rest['logf2'] > 0)]
        dict_ct_pos_DEG_but_not_marker[name_cell_types[i]] = pos_DEGs_but_no_mg
        i = i + 1
   
    results = {'DEGs': dict_df_ct_vs_rest, 'markers': dict_ct_markers, 'pos_DEGs_but_not_marker': dict_ct_pos_DEG_but_not_marker}
    return results

In [ ]:
def Jaccard(list1,list2):
    list1 = np.ceil(list1)
    list2 = np.ceil(list2)
    # list1 AND list2
    list3 = [list1[i] and list2[i] for i in range(len(list1))]
    list4 = [list1[i] or list2[i] for i in range(len(list1))]
    Jaccard = np.sum(list3)/np.sum(list4)
    return np.round(Jaccard,3)

In [2]:
def Jaccard_similarity_matrix(path_mg):
    df_mg = pd.read_csv(path_mg,index_col=0)
    Jaccard_sim = pd.DataFrame(index=df_mg.columns, columns=df_mg.columns)
    for i in df_mg.columns:
        for j in df_mg.columns:
            Jaccard_sim.loc[i,j] = Jaccard(df_mg[i].to_list(),df_mg[j].to_list())
    Jaccard_sim = Jaccard_sim.astype(float)
    sns.heatmap(Jaccard_sim,annot=True)
    print(df_mg.sum(axis=0))

In [ ]:
def Apply_strategy_1(adata,cell_types,leiden_clusters,path_mg)->dict:
    n_ct = len(cell_types)
    marker_gene_drop = {}
    for i in range(n_ct):
        for j in range(i+1,n_ct):
            d = DEGs_between_2_sets_leiden_clusters_compared_to_markers(adata,cell_types[i],leiden_clusters[i],cell_types[j],leiden_clusters[j],path_mg)
            if len(d['drop_'+cell_types[i]])>0:
                if cell_types[i] in marker_gene_drop:
                    for g in d['drop_'+cell_types[i]]:
                        if g not in marker_gene_drop[cell_types[i]]:
                            marker_gene_drop[cell_types[i]].append(g)
                else:
                    marker_gene_drop[cell_types[i]] = d['drop_'+cell_types[i]]
            if len(d['drop_'+cell_types[j]])>0:
                if cell_types[j] in marker_gene_drop:
                    for g in d['drop_'+cell_types[j]]:
                        if g not in marker_gene_drop[cell_types[j]]:
                            marker_gene_drop[cell_types[j]].append(g)
                else:
                    marker_gene_drop[cell_types[j]] = d['drop_'+cell_types[j]]
    print('Summary:')
    for key in marker_gene_drop.keys():
        print(key)
        print('Maybe drop:'+str(marker_gene_drop[key]))
    return marker_gene_drop

In [ ]:
def Apply_strategy_2(adata,cell_types,leiden_clusters,path_mg)->dict:
    dict_DEGs = DEGs_between_each_leiden_cluster_and_rest_compared_to_markers(adata,cell_types,leiden_clusters,path_mg)
    df_mg = pd.read_csv(path_mg,index_col=0)
    genes = adata.var_names
    marker_genes = df_mg.index.tolist()
    cell_types = ['Hepa','Kupf','LSEC','Endo vein','Fibr','Stel','Meso','Chol','B cell']
    marker_gene_add = {}
    for gene in genes:
        candidates = []
        for i in cell_types:
            if gene in dict_DEGs['pos_DEGs_but_not_marker'][i]['gene'].tolist():
                candidates.append(i)
                if i in marker_gene_add:
                    marker_gene_add[i].append(gene)
                else:
                    marker_gene_add[i] = [gene]
        if(len(candidates) > 0):
            print(gene)
            if gene in marker_genes:
                a = df_mg.loc[gene,:]
                b = a[a==1].index.values
                print('Is marker gene of: '+str(b.tolist()))
                print('Could also be a marker gene of: '+str(candidates))
            else:
                print('Is marker gene of: []')
                print('Could also be a marker gene of: '+str(candidates))
    print('Summary:')
    for key in marker_gene_add.keys():
        print(key)
        print('Maybe add:'+str(marker_gene_add[key]))
    return marker_gene_add

In [2]:
def Apply_strategy_multiple_times(adata,cell_types,leiden_clusters,path_mg,N,n_cells,strategy):
    results_runs = {}
    for k in range(N):
        list_sub_anndata = []
        for i in range(len(cell_types)):
            leiden_cl = [str(x) for x in leiden_clusters[i]]
            leiden = adata[adata.obs['leiden'].isin(leiden_cl),:]
            leiden_cells = leiden.obs.index.to_list()
            leiden_cells_random = sample(leiden_cells,n_cells)
            leiden = leiden[leiden_cells_random,:]
            list_sub_anndata.append(leiden)
        sub_anndata = ad.concat(list_sub_anndata)
        if strategy == 1:
            results_runs[k] = Apply_strategy_1(sub_anndata,cell_types,leiden_clusters,path_mg)
        if strategy == 2:
            results_runs[k] = Apply_strategy_2(sub_anndata,cell_types,leiden_clusters,path_mg)

    counts_run = {}
    for key in results_runs.keys():
        for k in results_runs[key].keys():
            if k not in counts_run.keys():
                counts_run[k] = results_runs[key][k]
            else:
                for i in results_runs[key][k]:
                    counts_run[k].append(i)
    final = {}
    for key in counts_run:
        df = pd.DataFrame(counts_run[key]).value_counts()
        # keep rows with value > N/2
        print(df)
        df = df[df > N/2]
        final[key] = df.index.to_list()

    for key in final.keys():
        final[key] = [i[0] for i in final[key]]

    return final